## Entendendo como extrair informações sobre repositórios no github

[Search](https://developer.github.com/v3/search/)
[Searching for repositories](https://help.github.com/en/articles/searching-for-repositories#search-by-repository-name-description-or-contents-of-the-readme-file)


Lista de qualifiers:  https://help.github.com/en/articles/searching-code
Documentação para search: https://developer.github.com/v3/search/

In [12]:
import requests
import pandas as pd
import time

Montando url de busca para os repositórios com maior quantidade de estrelas.
Usando como consulta stars:>1 para indicar que são todos os repositorios acima de 1 estrela, o que não tem impacto negativo para a busca pois existem certa de 3.305.358 repositórios e só vamos analisar 2.500.

In [2]:
q = 'q=stars:>1'
sort = '&sort=stars&order=desc'
url_base = 'https://api.github.com/search/repositories?'
url_final = url_base+q+sort
url_final

'https://api.github.com/search/repositories?q=stars:>1&sort=stars&order=desc'

In [13]:
colunas=['owner_type', 
           'owner_url',
           'owner_html_url',
           'html_url',
           'url',
           'fork',
           'created_at',
           'updated_at',
           'size',
           'stargazers_count',
           'language',
           'has_issues',
           'has_wiki',
           'forks_count',
           'forks',
           'open_issues',
           'watchers']

In [14]:
resultados = pd.DataFrame(columns=['owner_type', 
                                   'owner_url',
                                   'owner_html_url',
                                   'html_url',
                                   'url',
                                   'fork',
                                   'created_at',
                                   'updated_at',
                                   'size',
                                   'stargazers_count',
                                   'language',
                                   'has_issues',
                                   'has_wiki',
                                   'forks_count',
                                   'forks',
                                   'open_issues',
                                   'watchers',
                                   'commits',
                                   'contributors',
                                   'readme',])
resultados.head()

,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,stargazers_count,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme


In [15]:
def add_resultado(item):
    df = pd.DataFrame([[
                        item.get('owner').get('type', None),
                        item.get('owner').get('url', None),
                        item.get('owner').get('html_url', None),
                        item.get('html_url', None),
                        item.get('url', None),
                        item.get('fork', None),
                        item.get('created_at', None),
                        item.get('updated_at', None),
                        item.get('size', None),
                        item.get('stargazers_count', None),
                        item.get('language', None),
                        item.get('has_issues', None),
                        item.get('has_wiki', None),
                        item.get('forks_count', None),
                        item.get('forks', None),
                        item.get('open_issues', None),
                        item.get('watchers', None)
                    ]], columns=colunas)

    return df    

In [16]:
def extrair_dados(data, resultados):
    for item in data.get('items', None):
        resultados = pd.concat([resultados, add_resultado(item)], ignore_index=True, sort=False)
        
    return resultados

In [17]:
def get_total_paginas(data):
    
    itens_por_pagina = len(data.get('items'))
    total_paginas = data.get('total_count') // itens_por_pagina # opreração de div em python
    total_paginas

    print('Total de registros:{0} , Registros por página:{1}, Total de Páginas:{2}'.format(
        data.get('total_count'), itens_por_pagina, total_paginas))
    
    return total_paginas        

In [82]:
def percorrendo_paginas(url):
    
    resultados = pd.DataFrame(columns=['owner_type', 
                                   'owner_url',
                                   'owner_html_url',
                                   'html_url',
                                   'url',
                                   'fork',
                                   'created_at',
                                   'updated_at',
                                   'size',
                                   'stargazers_count',
                                   'language',
                                   'has_issues',
                                   'has_wiki',
                                   'forks_count',
                                   'forks',
                                   'open_issues',
                                   'watchers',
                                   'commits',
                                   'contributors',
                                   'readme',])
    
    print('Extraindo página:1')
    results = requests.get(url)    
    data = dict(results.json())
    resultados = extrair_dados(data, resultados)

    # iteracoes = get_total_paginas(data)
    iteracoes = 84 # Para pegar em torno de 2500 registros, semelhante ao artigo original
    
    for iteracao in range(1, 34):
        
        # Para requisições não autenticadas a api restringe para 10 requisições por minuto,
        # Para requisições autenticadas 30 por minuto.
        if iteracao in [10, 20, 30, 40, 50, 60, 70, 80]:
            print("sleep 1 minuto")
            time.sleep(60)
        
        header = dict(results.links)
        next_url = header.get('next').get('url')
        print("----------> next url: {0}".format(next_url))
        
        print('Extraindo página:{0}'.format(iteracao+1))
        results = requests.get(next_url)
        print('Status:{0}'.format(results))
        
        data = dict(results.json())
        print('Data keys:{0}'.format(data.keys()))
            
        resultados = extrair_dados(data, resultados)
        
    return resultados

In [83]:
%%time
resultados = percorrendo_paginas(url_final)

Extraindo página:1
----------> next url: https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=2
Extraindo página:2
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
----------> next url: https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=3
Extraindo página:3
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
----------> next url: https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=4
Extraindo página:4
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
----------> next url: https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=5
Extraindo página:5
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
----------> next url: https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc

In [85]:
resultados.describe()

,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,stargazers_count,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme
count,1020,1020,1020,1020,1020,1020,1020,1020,1020,1020,899,1020,1020,1020,1020,1020,1020,0.0,0.0,0.0
unique,2,793,793,936,936,1,936,913,924,908,39,2,2,881,881,456,908,0.0,0.0,0.0
top,Organization,https://api.github.com/users/facebook,https://github.com/facebook,https://github.com/portainer/portainer,https://api.github.com/repos/requirejs/requirejs,False,2009-10-04T23:41:19Z,2019-07-01T15:24:29Z,1416,13179,JavaScript,True,True,1719,1719,2,13179,NaN,NaN,NaN
freq,620,19,19,2,2,1020,2,3,3,4,327,975,687,4,4,20,4,NaN,NaN,NaN


In [87]:
resultados.to_csv('../dados/repositorios.csv', index=False)

In [88]:
resultados.columns.values

array(['owner_type', 'owner_url', 'owner_html_url', 'html_url', 'url',
       'fork', 'created_at', 'updated_at', 'size', 'stargazers_count',
       'language', 'has_issues', 'has_wiki', 'forks_count', 'forks',
       'open_issues', 'watchers', 'commits', 'contributors', 'readme'],
      dtype=object)